In [1]:
from sklearn.cluster import KMeans
import pandas as pd
import csv
import numpy as np
from sklearn.mixture import GaussianMixture
import socket
import csv
import operator


data_both1 = np.genfromtxt(
    '24-03-17/akkibotheye1.csv',           # file name
    skip_header=0,          # lines to skip at the top
    skip_footer=0,          # lines to skip at the bottom
    delimiter=',',          # column delimiter
    dtype='float32'        # data type
    )

data_left5 = np.genfromtxt(
    '03-04-17/akkilefteye5.csv',           # file name
    skip_header=0,          # lines to skip at the top
    skip_footer=0,          # lines to skip at the bottom
    delimiter=',',          # column delimiter
    dtype='float32'        # data type
    )

data_right7 = np.genfromtxt(
    '03-04-17/akkirighteye7.csv',           # file name
    skip_header=0,          # lines to skip at the top
    skip_footer=0,          # lines to skip at the bottom
    delimiter=',',          # column delimiter
    dtype='float32'        # data type
    )

data_jaw3 = np.genfromtxt(
    '24-03-17/akkijaw3.csv',           # file name
    skip_header=0,          # lines to skip at the top
    skip_footer=0,          # lines to skip at the bottom
    delimiter=',',          # column delimiter
    dtype='float32'        # data type
    )


X_both1 = data_both1[0:3000, 2:5]
X_left5 = data_left5[0:3000, 2:5]
X_right7 = data_right7[0:3000, 2:5]
X_jaw3 = data_jaw3[0:3000, 3:7]

kmeans_both1 = KMeans(n_clusters=2, random_state=0).fit(X_both1)
kmeans_left5 = KMeans(n_clusters=2, random_state=0).fit(X_left5)
kmeans_right7 = KMeans(n_clusters=2, random_state=0).fit(X_right7)
kmeans_jaw3 = KMeans(n_clusters=2, random_state=0).fit(X_jaw3)
print "Both Clusters Point"
print kmeans_both1.cluster_centers_
print "Left Clusters Point"
print kmeans_left5.cluster_centers_
print "Right Clusters Point"
print kmeans_right7.cluster_centers_
print "Jaw Clusters Point"
print kmeans_jaw3.cluster_centers_

Both Clusters Point
[[  2.00158119   2.82148385   3.50001335]
 [ 14.40033913  22.40860367  27.60883331]]
Left Clusters Point
[[  1.31832981   1.80516541   2.39351892]
 [  9.18116188  13.67080402  17.61940384]]
Right Clusters Point
[[  1.26736856   1.60145831   1.83342838]
 [ 15.13414097  19.49867249  20.34691048]]
Jaw Clusters Point
[[  1.2586385    1.58546984   1.8742094    1.96541357]
 [  7.39645004   9.0999527   10.73330879  10.99831009]]


In [ ]:
import time
import serial
import warnings
import os
warnings.filterwarnings('ignore')
UDP_IP = "127.0.0.1"
UDP_PORT = 12345
count = 0
port = "COM5"
baud = 2400
ser = serial.Serial(port, baud, timeout = 3)
sock = socket.socket(socket.AF_INET,socket.SOCK_DGRAM)
sock.bind((UDP_IP, UDP_PORT))
#fo = open("ashmittest.csv", "wb")
status = 1
condition = 0
pre5 = 0
pre7 = 0
flag = 1


while True:
    count=count+1
    data, addr = sock.recvfrom(1024)
    data_new = data[1:-1]
    data_X = data_new.split(',')
    data_X = np.array(data_X)
    
    data, addr = sock.recvfrom(1024)
    data_new = data[1:-1]
    data_A = data_new.split(',')
    data_A = np.array(data_A)  
    
    data, addr = sock.recvfrom(1024)
    data_new = data[1:-1]
    data_Y = data_new.split(',')
    data_Y = np.array(data_Y)
    
    data, addr = sock.recvfrom(1024)
    data_new = data[1:-1]
    data_Z = data_new.split(',')
    data_Z = np.array(data_Z)
    

    
    predicted_both1 = kmeans_both1.predict(data_X[2:5])
    predicted_left5 = kmeans_left5.predict(data_Y[2:5])
    predicted_right7 = kmeans_right7.predict(data_Z[2:5])
    predicted_jaw3 = kmeans_jaw3.predict(data_A[3:7])
    
    if(predicted_jaw3 == 1 and status == 1):
        os.system('cls' if os.name == 'nt' else 'clear')
        print str(count) + " jaw clench"
        
        ser.write('S')
        status = 0 
        
    elif(predicted_both1 == 1 and status == 1):
        os.system('cls' if os.name == 'nt' else 'clear')
        print str(count) + "  both Eye Blink"
        ser.write('W')
        status = 0
    
    elif(status == 1 and (pre5 > predicted_left5 or pre7 > predicted_right7)):
        if(condition == 2):
            os.system('cls' if os.name == 'nt' else 'clear')
            print str(count)+ " Right Eye Blink"
            ser.write('D')
        if(condition == 3):
            os.system('cls' if os.name == 'nt' else 'clear')
            print str(count) + " Left Eye Blink"
            ser.write('A')
        if(condition == 4):
            os.system('cls' if os.name == 'nt' else 'clear')
            print str(count) + " Normal EYE BLINK"
        pre5 = 0
        pre7 = 0
        condition = 0
    ##
    ##if(predicted_both1 == 0 and predicted_left5 == 0 and predicted_right7 == 1  and status = 1):
    ##    condition = 2
        
    ##    
    if(predicted_both1 == 0 and predicted_left5 == 0 and predicted_right7 == 1  ):
        condition = 2
        pre7 = predicted_right7
        
        #print str(count) + "  right Eye Blink"
        #status = 0;
    
    if (predicted_both1 == 0 and predicted_left5 == 1 and predicted_right7 == 0 ):
        condition = 3
        pre5 = predicted_left5
        
        #print str(count) + "  both Eye Blink"
        #status = 0;
      
    if (predicted_both1 == 0 and predicted_left5 == 1 and data_Z[3:4] > 1.5):
        condition = 4   
        
    if(predicted_both1 == 0 and predicted_left5 == 0 and predicted_right7 == 0 and predicted_jaw3 == 0 and status == 0 ):
        status = 1;
        pre5 = 0
        pre7 = 0
        condition = 0
       
        
     
    
    
        
sock.close();   

    

140 Left Eye Blink
156 Left Eye Blink
216  both Eye Blink
263 Left Eye Blink
284 jaw clench
382  both Eye Blink
456 jaw clench
1121  both Eye Blink
1210 Left Eye Blink
1308  both Eye Blink
1371 Right Eye Blink
1430 Right Eye Blink
1497 Left Eye Blink
1503 jaw clench
1692 jaw clench
2153  both Eye Blink
2326  both Eye Blink
2363 jaw clench
2636  both Eye Blink
2668 jaw clench
2784 Right Eye Blink
2835 jaw clench
2915 Left Eye Blink
2928 jaw clench
2968  both Eye Blink
3013 jaw clench
3079 Right Eye Blink
3139 jaw clench
3253 jaw clench
3300  both Eye Blink
3339 jaw clench
3473 jaw clench
3523 Left Eye Blink
3579  both Eye Blink
3675 jaw clench
3766 jaw clench
4087 jaw clench
4562  both Eye Blink
4906  both Eye Blink
4978 Left Eye Blink
5161  both Eye Blink
